# 딥러닝 3가지 **분류** 예제 (최종: 쿠팡 해지 예상 고객 예측) — 표준화 없음

요청사항: **표준화(평균/표준편차로 스케일링) 없이** 3가지 분류 문제를 모두 실행 가능하게 구성했습니다.

공통 흐름:
1) CSV 준비/로드  
2) 모델 정의  
3) 학습  
4) 평가/예측

구성:
- 예제 1: **이진 분류(로지스틱)** — 입력 1개(단일 컬럼)
- 예제 2: **다중 클래스 분류(3클래스)** — 입력 2개(2컬럼)
- 최종: **쿠팡 해지(Churn) 이진 분류(MLP)** — 입력 6개

> 표준화를 빼면 학습률(lr) 튜닝이 더 중요해집니다.  
> 이 노트북은 **발산을 피하도록 lr을 보수적으로 설정**했습니다.

In [11]:
# =========================
# 0) 환경 준비
# =========================
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

np.random.seed(42)
torch.manual_seed(42)

print("torch:", torch.__version__)

torch: 2.9.0+cu126


## 예제 1) 이진 분류(입력 1개): 앱 이용점수로 “구독 유지/해지 위험” 분류

### 문제
CSV(`app_subscribe.csv`)에는 **이용점수(engagement_score)** 와 **구독 유지 여부(subscribe=1/0)** 가 있습니다.  
PyTorch로 가장 단순한 **로지스틱 분류(Linear 1→1)** 를 학습하고, 점수 35/75의 구독 확률을 예측하세요.

- 입력: `engagement_score` (1개)
- 라벨: `subscribe` (0/1)
- 손실: `BCEWithLogitsLoss`
- 평가지표: Accuracy(0.5 threshold)

In [13]:
# 1) CSV 생성
csv_text = """engagement_score,subscribe
10,0
15,0
20,0
25,0
30,0
35,0
40,1
50,1
60,1
70,1
80,1
90,1
"""
with open("app_subscribe.csv", "w", encoding="utf-8") as f:
    f.write(csv_text)

# 2) 로드
df = pd.read_csv("app_subscribe.csv")
X = torch.tensor(df[["engagement_score"]].values, dtype=torch.float32)  # (N,1)
y = torch.tensor(df[["subscribe"]].values, dtype=torch.float32)         # (N,1)

# 3) 모델(로지스틱): logits 출력
model = nn.Linear(1, 1)
loss_fn = nn.BCEWithLogitsLoss()

# 표준화 없이 학습률을 보수적으로(발산 방지)
opt = torch.optim.SGD(model.parameters(), lr=0.001)

# 4) 학습
for epoch in range(6000):
    logits = model(X)
    loss = loss_fn(logits, y)
    opt.zero_grad()
    loss.backward()
    opt.step()

# 5) 평가/예측
with torch.no_grad():
    prob = torch.sigmoid(model(X))
    pred = (prob >= 0.5).float()
    acc = (pred == y).float().mean().item()

    test = torch.tensor([[35.0],[75.0]], dtype=torch.float32)
    test_prob = torch.sigmoid(model(test))

print("Train ACC:", acc)
print("engagement_score=35 구독확률:", float(test_prob[0]))
print("engagement_score=75 구독확률:", float(test_prob[1]))

Train ACC: 0.5833333134651184
engagement_score=35 구독확률: 0.6328263282775879
engagement_score=75 구독확률: 0.8311929702758789


## 예제 2) 다중 클래스 분류(3클래스, 입력 2개): 날씨로 옷차림 추천 분류

### 문제
CSV(`outfit_weather.csv`)에는 **기온(temp_c)** 과 **풍속(wind_mps)** 로부터 옷차림 클래스를 예측하는 데이터가 있습니다.

- 라벨 `outfit_class`
  - 0: 얇게(반팔/얇은 겉옷)
  - 1: 보통(긴팔/가벼운 아우터)
  - 2: 두껍게(패딩/두꺼운 옷)

PyTorch로 **3클래스 분류 모델**을 학습하고, 아래 입력의 클래스를 예측하세요.
- (temp=22, wind=2)
- (temp=8, wind=5)
- (temp=15, wind=3)

> 다중 분류는 `CrossEntropyLoss`를 사용하며, 라벨은 **정수(0,1,2)** 여야 합니다.

In [14]:
# 1) CSV 생성
csv_text = """temp_c,wind_mps,outfit_class
28,1,0
26,2,0
24,3,0
22,1,0
21,3,1
20,4,1
18,2,1
17,4,1
16,3,1
14,2,1
13,4,2
12,5,2
10,3,2
9,4,2
7,6,2
"""
with open("outfit_weather.csv", "w", encoding="utf-8") as f:
    f.write(csv_text)

# 2) 로드
df = pd.read_csv("outfit_weather.csv")
X = torch.tensor(df[["temp_c", "wind_mps"]].values, dtype=torch.float32)  # (N,2)
y = torch.tensor(df["outfit_class"].values, dtype=torch.long)            # (N,) 0/1/2

# 3) train/val split(작은 데이터라 간단히 랜덤)
N = len(df)
idx = torch.randperm(N)
train_size = int(N * 0.8)
train_idx = idx[:train_size]
val_idx = idx[train_size:]

X_train, y_train = X[train_idx], y[train_idx]
X_val, y_val     = X[val_idx], y[val_idx]

# 4) 모델(가장 단순: Linear 2->3)
model = nn.Linear(2, 3)  # logits 3개
loss_fn = nn.CrossEntropyLoss()

# 표준화 없이 lr은 조금 보수적으로
opt = torch.optim.SGD(model.parameters(), lr=0.05)

# 5) 학습
for epoch in range(2000):
    logits = model(X_train)
    loss = loss_fn(logits, y_train)
    opt.zero_grad()
    loss.backward()
    opt.step()

# 6) 평가/예측
with torch.no_grad():
    val_logits = model(X_val)
    val_pred = torch.argmax(val_logits, dim=1)
    val_acc = (val_pred == y_val).float().mean().item()

    test = torch.tensor([[22.0, 2.0],
                         [ 8.0, 5.0],
                         [15.0, 3.0]], dtype=torch.float32)
    test_logits = model(test)
    test_pred = torch.argmax(test_logits, dim=1)

print("Val ACC:", val_acc)
print("예측 클래스(0/1/2):", test_pred.tolist())

Val ACC: 1.0
예측 클래스(0/1/2): [0, 2, 1]


### 클래스 해석
- 0: 얇게  
- 1: 보통  
- 2: 두껍게

## 최종 문제) 딥러닝 - 쿠팡 해지 예상 고객(Churn) **이진 분류**

### 문제 설명
쿠팡(또는 구독/커머스 서비스)에서 **향후 30일 내 해지(1) / 유지(0)** 를 예측한다고 가정합니다.

CSV(`coupang_churn.csv`) 컬럼:
- 입력(독립변수)
  - `tenure_months` : 가입 후 경과 개월
  - `orders_30d` : 최근 30일 주문 수
  - `avg_spend` : 최근 30일 평균 결제금액(가상 단위)
  - `cancel_rate` : 최근 취소 비율(0~1)
  - `cs_tickets_30d` : 최근 30일 고객센터 문의 수
  - `discount_use_rate` : 할인/쿠폰 사용 비율(0~1)
- 라벨(종속변수)
  - `churn_30d` : 30일 내 해지 여부(0/1)

### 목표
1) 훈련/검증 분리  
2) 간단 MLP로 학습  
3) 전체 고객의 해지 확률을 구해 **상위 5명** 출력

> 표준화를 안 쓰면 `avg_spend`처럼 스케일 큰 피처가 학습에 영향을 더 줄 수 있습니다.  
> 여기서는 Adam과 보수적 학습률로 안정성을 확보합니다.

In [16]:
# =========================
# 1) CSV 생성(실습용) - 실무라면 실데이터로 교체
# =========================
np.random.seed(42)

N = 500
tenure = np.random.randint(1, 60, size=N)
orders = np.random.poisson(lam=6, size=N).clip(0, 30)
avg_spend = (np.random.normal(40, 15, size=N)).clip(5, 120)
cancel_rate = np.random.beta(a=2, b=8, size=N)
cs = np.random.poisson(lam=1.2, size=N).clip(0, 10)
discount = np.random.beta(a=2.5, b=4, size=N)

# 해지 확률 설계(학습 가능하도록 단순 구조)
logit = (
    -0.03 * tenure
    -0.10 * orders
    -0.01 * avg_spend
    + 2.8  * cancel_rate
    + 0.30 * cs
    + 1.3  * discount
    - 0.1
)
prob = 1 / (1 + np.exp(-logit))
churn = (np.random.rand(N) < prob).astype(int)

df = pd.DataFrame({
    "customer_id": np.arange(1, N+1),
    "tenure_months": tenure,
    "orders_30d": orders,
    "avg_spend": np.round(avg_spend, 2),
    "cancel_rate": np.round(cancel_rate, 4),
    "cs_tickets_30d": cs,
    "discount_use_rate": np.round(discount, 4),
    "churn_30d": churn
})
df.to_csv("coupang_churn.csv", index=False, encoding="utf-8")
df.head()

,customer_id,tenure_months,orders_30d,avg_spend,cancel_rate,cs_tickets_30d,discount_use_rate,churn_30d
0,1,39,2,32.97,0.2906,2,0.2307,1
1,2,52,4,22.68,0.1561,0,0.2693,1
2,3,29,3,23.57,0.1974,2,0.4031,0
3,4,15,5,44.48,0.3113,2,0.4955,1
4,5,43,9,39.17,0.1083,3,0.4507,0


In [18]:
# =========================
# 2) 로드/전처리 (표준화 없음)
# =========================
df = pd.read_csv("coupang_churn.csv")

feature_cols = [
    "tenure_months",
    "orders_30d",
    "avg_spend",
    "cancel_rate",
    "cs_tickets_30d",
    "discount_use_rate"
]
target_col = "churn_30d"

X = torch.tensor(df[feature_cols].values, dtype=torch.float32)
y = torch.tensor(df[[target_col]].values, dtype=torch.float32)  # (N,1) 0/1

# train/val split
N = len(df)                    # 전체 데이터 개수(N)
idx = torch.randperm(N)        # 0~N-1 인덱스를 무작위로 섞기
train_size = int(N * 0.8)      # 학습용 개수(80%)
train_idx = idx[:train_size]   # 섞인 인덱스 중 앞 80% = 학습 인덱스
val_idx = idx[train_size:]     # 나머지 20% = 검증 인덱스

X_train, y_train = X[train_idx], y[train_idx]  # 학습 데이터(입력/정답)
X_val, y_val     = X[val_idx], y[val_idx]      # 검증 데이터(입력/정답)


print("Train/Val:", X_train.shape, X_val.shape)
print("Churn rate(train):", float(y_train.mean()))

Train/Val: torch.Size([400, 6]) torch.Size([100, 6])
Churn rate(train): 0.36500000953674316


In [20]:
# =========================
# 3) 모델(간단 MLP) + 학습
# =========================

# 여러 층을 순서대로 쌓은 신경망(MLP)
model = nn.Sequential(
    nn.Linear(len(feature_cols), 16),  # 입력(특성개수) -> 16개 뉴런
    nn.ReLU(),                         # 음수는 0으로, 비선형 추가
    nn.Linear(16, 8),                  # 16 -> 8개 뉴런
    nn.ReLU(),                         # 비선형 추가
    nn.Linear(8, 1)                    # 8 -> 1개 출력(점수=logits)
)

loss_fn = nn.BCEWithLogitsLoss()       # 이진분류 손실(시그모이드 포함)
opt = torch.optim.Adam(model.parameters(), lr=0.001)  # 가중치 업데이트(Adam)

for epoch in range(400):               # 400번 학습 반복
    logits = model(X_train)            # 예측 점수(logits) 계산
    loss = loss_fn(logits, y_train)    # 손실 계산(예측 vs 정답)

    opt.zero_grad()                    # 이전 기울기(grad) 초기화
    loss.backward()                    # 기울기 계산(역전파)
    opt.step()                         # 가중치 업데이트

    if (epoch + 1) % 40 == 0:          # 40번마다 검증 성능 출력
        with torch.no_grad():          # 평가할 땐 grad 계산 안 함
            v_logits = model(X_val)    # 검증 예측 점수
            v_prob = torch.sigmoid(v_logits)          # 점수 -> 확률(0~1)
            v_pred = (v_prob >= 0.5).float()          # 0.5 기준으로 0/1 예측
            v_acc = (v_pred == y_val).float().mean().item()  # 정확도
            v_loss = loss_fn(v_logits, y_val).item()         # 검증 손실
        print(f"epoch {epoch+1:3d} | train_loss {loss.item():.4f} | val_loss {v_loss:.4f} | val_acc {v_acc:.3f}")


epoch  40 | train_loss 0.6296 | val_loss 0.6545 | val_acc 0.620
epoch  80 | train_loss 0.6216 | val_loss 0.6546 | val_acc 0.630
epoch 120 | train_loss 0.6195 | val_loss 0.6525 | val_acc 0.630
epoch 160 | train_loss 0.6172 | val_loss 0.6518 | val_acc 0.620
epoch 200 | train_loss 0.6147 | val_loss 0.6504 | val_acc 0.620
epoch 240 | train_loss 0.6120 | val_loss 0.6495 | val_acc 0.620
epoch 280 | train_loss 0.6092 | val_loss 0.6487 | val_acc 0.610
epoch 320 | train_loss 0.6062 | val_loss 0.6473 | val_acc 0.600
epoch 360 | train_loss 0.6025 | val_loss 0.6459 | val_acc 0.600
epoch 400 | train_loss 0.5984 | val_loss 0.6458 | val_acc 0.600


In [23]:
# 한 사람 입력(특성 6개 순서: feature_cols와 동일해야 함)
one = torch.tensor([[12.0, 3.0, 35.0, 0.20, 1.0, 0.40]], dtype=torch.float32)
# 예: tenure_months=12, orders_30d=3, avg_spend=35, cancel_rate=0.20, cs_tickets_30d=1, discount_use_rate=0.40

with torch.no_grad():
    logit = model(one).item()        # 모델 점수(확률 아님)
    churn = 1 if logit >= 0 else 0   # 0 이상이면 해지(1), 아니면 유지(0)

print("logit:", logit)
print("예측:", "해지(1)" if churn == 1 else "유지(0)")


logit: -0.03167670965194702
예측: 유지(0)


In [ ]:
# =========================
# 4) 전체 고객 해지 확률 예측 + 상위 5명 출력
# =========================
with torch.no_grad():                                 # 예측할 땐 학습(grad) 안 함
    churn_prob = torch.sigmoid(model(X)).squeeze(1).numpy()
    # 모델 출력(logits) -> 확률(0~1)로 변환, (N,1)->(N,)로 만들고 numpy로 변환

df_out = df[["customer_id"] + feature_cols + [target_col]].copy()  # 필요한 컬럼만 복사
df_out["churn_prob"] = churn_prob                                  # 해지 확률 컬럼 추가

top5 = df_out.sort_values("churn_prob", ascending=False).head(5)   # 확률 높은 순으로 5명 뽑기


### (선택) 실무 체크포인트(최소)
- 데이터 기간 정의(관측창/예측창)와 타깃 정의가 먼저입니다.
- 이진 분류에서는 Accuracy 외에도 **ROC-AUC / PR-AUC**를 자주 봅니다.


## 전체 요약 표

In [15]:
summary = pd.DataFrame([
    {
        "예제": "예제 1",
        "분류 종류": "이진(0/1)",
        "입력 컬럼": "engagement_score(1개)",
        "모델": "Linear(1->1) + Sigmoid(확률)",
        "손실": "BCEWithLogitsLoss",
        "출력": "구독 확률"
    },
    {
        "예제": "예제 2",
        "분류 종류": "다중(3클래스)",
        "입력 컬럼": "temp_c, wind_mps(2개)",
        "모델": "Linear(2->3)",
        "손실": "CrossEntropyLoss",
        "출력": "클래스(0/1/2)"
    },
    {
        "예제": "최종",
        "분류 종류": "이진(0/1)",
        "입력 컬럼": "고객 피처 6개",
        "모델": "MLP(16-8-1)",
        "손실": "BCEWithLogitsLoss",
        "출력": "해지 확률 + 상위 5명"
    }
])
summary

,예제,분류 종류,입력 컬럼,모델,손실,출력
0,예제 1,이진(0/1),engagement_score(1개),Linear(1->1) + Sigmoid(확률),BCEWithLogitsLoss,구독 확률
1,예제 2,다중(3클래스),"temp_c, wind_mps(2개)",Linear(2->3),CrossEntropyLoss,클래스(0/1/2)
2,최종,이진(0/1),고객 피처 6개,MLP(16-8-1),BCEWithLogitsLoss,해지 확률 + 상위 5명
